# Teste de conexão com o banco de dados 

In [ ]:
import pandas as pd
import oracledb
from tabulate import tabulate

# this is the SQL text that will be executed
sql_text = """
SELECT * FROM RM99210.USUARIOS
"""

# here we create a python connection,cursor object and execute the statement using  Cursor.execute().
# connections and cursors should be released when they are no longer needed.
# closed automatically when the variable referencing it goes out of scope (and no further references are retained).
# `with` block is a convenient way to ensure this.
with oracledb.connect(
  user = 'RM99667',
  password = '260903',
  dsn = 'oracle.fiap.com.br:1521/ORCL',
  config_dir = 'C:\\Users\\victo\\Oracle\\network\\admin',
) as connection:
    with connection.cursor() as cursor:
        # Fetch all rows from the executed query
        rows = cursor.execute(sql_text).fetchall()
        # Fetch column names
        column_names = [col[0] for col in cursor.description]

# Print the results in a tabulated format
print(tabulate(rows, headers=column_names, tablefmt='grid'))

print("Done!!")

# Deletar usuario 


In [ ]:
import ipywidgets as widgets
from IPython.display import display
import oracledb

# Widget para inserir o ID do usuário a ser deletado
id_usuario = widgets.IntText(description='ID Usuário:')

# Função para deletar o usuário do banco de dados
def deletar_usuario(b):
    with oracledb.connect(
        user = 'RM99667',
        password = '260903',
        dsn = 'oracle.fiap.com.br:1521/ORCL',
        config_dir = 'C:\\Users\\victo\\Oracle\\network\\admin',
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute("""
                DELETE FROM RM99210.USUARIOS
                WHERE ID_USUARIO = :1
            """, (id_usuario.value,))
            connection.commit()
    print(f"Usuário com ID {id_usuario.value} deletado com sucesso!")

# Botão para acionar a deleção
botao_deletar = widgets.Button(description='Deletar Usuário')
botao_deletar.on_click(deletar_usuario)

# Exibir widgets
display(id_usuario, botao_deletar)

# Cadastro de usuario

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import oracledb

# Widgets para cada campo da tabela
nm_usuario = widgets.Text(description='Nome Usuário:')
ds_email = widgets.Text(description='Email:')
ds_especialidade = widgets.Text(description='Especialidade:')
nr_nivel_experiencia = widgets.IntText(description='Nível Experiência:')
ds_localizacao = widgets.Text(description='Localização:')
dt_nascimento = widgets.DatePicker(description='Nascimento:')
ps_usuario = widgets.Textarea(description='Senha:')

# Função para hashear a senha
def hash_password(password):
    import hashlib
    return hashlib.sha256(password.encode()).hexdigest()

# Função para inserir os dados no banco de dados
def inserir_dados(b):
    with oracledb.connect(
        user='RM99667',
        password='260903',
        dsn='oracle.fiap.com.br:1521/ORCL',
        config_dir='C:\\Users\\victo\\Oracle\\network\\admin',
    ) as connection:
        with connection.cursor() as cursor:
            # Garantir que a data esteja no formato correto
            data_nascimento = dt_nascimento.value.strftime('%Y-%m-%d')
            # Hashear a senha
            hashed_password = hash_password(ps_usuario.value)
            cursor.execute("""
                INSERT INTO RM99210.USUARIOS (
                    ID_USUARIO, NM_USUARIO, DS_EMAIL, DS_ESPECIALIDADE,
                    NR_NIVEL_EXPERIENCIA, DS_LOCALIZACAO,
                    DT_NASCIMENTO, PS_USUARIO
                ) VALUES (
                    RM99210.SEQ_USUARIOS_ID.NEXTVAL, :1, :2, :3, :4, :5, TO_DATE(:6, 'YYYY-MM-DD'), :7
                )
            """, (
                nm_usuario.value, ds_email.value, ds_especialidade.value,
                nr_nivel_experiencia.value, ds_localizacao.value,
                data_nascimento, hashed_password
            ))
            connection.commit()
    print("Dados inseridos com sucesso!")

# Botão para acionar a inserção
botao_inserir = widgets.Button(description='Inserir Dados')
botao_inserir.on_click(inserir_dados)

# Exibir widgets
display(nm_usuario, ds_email, ds_especialidade, nr_nivel_experiencia, ds_localizacao, dt_nascimento, ps_usuario, botao_inserir)

## Login de usuario

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import oracledb
import hashlib

# Function to hash the password
def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

# Function to log in the user
def login(username, password):
    hashed_password = hash_password(password)
    with oracledb.connect(
        user='RM99667',
        password='260903',
        dsn='oracle.fiap.com.br:1521/ORCL',
        config_dir='C:\\Users\\victo\\Oracle\\network\\admin',
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute("""
                SELECT COUNT(*) FROM RM99210.USUARIOS
                WHERE NM_USUARIO = :1 AND PS_USUARIO = :2
            """, (username, hashed_password))
            result = cursor.fetchone()
            if result[0] > 0:
                print("Login successful!")
            else:
                print("Invalid username or password.")

# Widgets for username and password
username_widget = widgets.Text(description='Username:')
password_widget = widgets.Password(description='Password:')
login_button = widgets.Button(description='Login')

# Function to handle login button click
def on_login_button_clicked(b):
    username = username_widget.value
    password = password_widget.value
    login(username, password)

# Attach the click event to the login button
login_button.on_click(on_login_button_clicked)

# Display the widgets
display(username_widget, password_widget, login_button)